In [1]:
!pip install scikit-learn


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# 운영 체제와 경고 메시지 관련 설정입니다.
import os, warnings
warnings.filterwarnings("ignore")  # 코드 실행 중 불필요한 경고를 숨깁니다.

# 데이터 처리 및 인공지능 학습에 필요한 라이브러리를 불러옵니다.
import numpy as np        # 수치 계산을 위한 라이브러리입니다.
import pandas as pd       # 표 형태 데이터를 처리하는 라이브러리입니다.
import torch              # 인공지능 모델 학습을 위한 라이브러리입니다.
import torch.nn as nn     # 인공지능 모델의 계층을 구성하는 모듈입니다.
import torch.optim as optim  # 모델 학습 방법(최적화 방법)을 제공하는 모듈입니다.

# 데이터셋 분리 및 평가 지표 계산을 위한 라이브러리입니다.
from sklearn.model_selection import train_test_split  # 데이터를 훈련용과 평가용으로 나누는 함수입니다.
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score  # 성능 평가를 위한 지표입니다.

# 학습 효율을 높이기 위한 추가 라이브러리입니다.
from torch.optim.lr_scheduler import ReduceLROnPlateau   # 학습률을 자동으로 조정해주는 함수입니다.
from torch.nn.utils.rnn import pack_padded_sequence      # 문장 길이에 맞춰 효율적으로 처리하는 함수입니다.
from gensim.models.fasttext import load_facebook_model   # 단어를 숫자 벡터로 변환하는 모델입니다.
from konlpy.tag import Okt                               # 한글 문장을 형태소로 나누는 도구입니다.
from torch.utils.data import DataLoader, Dataset         # 데이터를 미니배치로 나눠주는 도구입니다.
from torch.utils import data
# 하이퍼파라미터(모델이 학습할 때 쓰는 여러 가지 옵션이나 설정값)를 정하는 부분입니다.
# 하이퍼파라미터는 사람이 미리 정하는 값으로, 인공지능 모델이 데이터를 얼마나 잘 학습할지에 큰 영향을 줍니다.

CSV_PATH = "customer_trial_requests.csv"
# CSV_PATH: 사용할 데이터가 저장된 CSV 파일의 이름입니다.
# 여기서는 'customer_trial_requests.csv' 라는 파일을 사용합니다.

FASTTEXT_BIN = "cc.ko.300.bin"
# FASTTEXT_BIN: FastText라는 모델을 불러오는 데 사용할 파일의 이름입니다.
# FastText는 단어를 컴퓨터가 이해할 수 있도록 숫자로 바꿔주는 모델입니다.
# 'cc.ko.300.bin' 파일은 미리 학습된 한글 단어 벡터를 담고 있습니다.

MAX_SEQ_LEN = 50
# MAX_SEQ_LEN: 한 문장에서 최대 몇 개의 단어까지 사용할지를 정하는 값입니다.
# 문장이 길면 앞부분을 자르고 뒤에서부터 50개의 단어만 사용합니다.
# 문장이 짧으면 남는 자리는 숫자 0으로 채워서 길이를 맞춥니다.

BATCH_SIZE = 16
# BATCH_SIZE: 한 번에 인공지능이 학습할 데이터의 개수입니다.
# 여기서는 16개의 데이터를 한 번에 묶어서 학습합니다.
# 이 값이 크면 한 번에 많은 데이터를 사용하니 빠르게 학습되지만 메모리를 더 많이 사용합니다.
# 작으면 학습 속도는 조금 느려질 수 있지만 메모리 적게 사용하고 더 세밀한 학습이 가능합니다.

EPOCHS = 100
# EPOCHS: 전체 데이터를 몇 번 반복해서 학습할지를 정하는 값입니다.
# 100으로 설정하면 전체 데이터를 100번 반복해서 학습합니다.
# 이 숫자가 너무 작으면 학습이 제대로 이루어지지 않고, 너무 크면 과도한 학습(과적합)이 될 수 있습니다.

SEED = 42
# SEED: 난수(랜덤한 숫자)를 생성할 때 사용하는 기준 숫자입니다.
# SEED 값을 설정하면 항상 같은 랜덤 값이 생성되어 매번 동일한 결과를 얻을 수 있습니다.
# 주로 실험 결과를 재현할 때 사용됩니다.

LR = 1e-3
# LR (Learning Rate, 학습률): 인공지능 모델이 학습할 때, 한 번에 얼마나 크게 가중치를 조정할지를 결정하는 값입니다.
# 1e-3 = 0.001과 같은 값으로, 아주 작은 양입니다.
# 이 값이 너무 크면 학습이 불안정해지고, 너무 작으면 학습 속도가 느려질 수 있습니다.

WEIGHT_DECAY = 1e-4
# WEIGHT_DECAY: 과도한 학습을 막기 위해 모델의 가중치(weight)에 약한 제한을 주는 값입니다.
# 가중치 값이 너무 커지는 것을 방지해서 모델이 새로운 데이터에도 잘 작동할 수 있게 도와줍니다.
# 1e-4 = 0.0001로, 작은 값입니다. 일반적으로 작은 값으로 설정합니다.

CLIP_NORM = 1.0
# CLIP_NORM: 학습 중에 그래디언트(기울기)가 너무 커지는 것을 막기 위해 제한하는 값입니다.
# 그래디언트가 너무 크면 학습이 불안정해질 수 있습니다.
# 1.0을 넘으면 자동으로 그래디언트 크기를 줄여 안정적으로 학습할 수 있게 합니다.

VAL_SIZE = 0.1
# VAL_SIZE: 학습 중에 모델이 얼마나 잘 학습하고 있는지 확인할 때 사용하는 데이터(검증 데이터)의 비율입니다.
# 0.1이면 전체 데이터의 10%를 모델 검증용으로 사용합니다.
# 모델의 성능을 정확히 평가하기 위해 사용됩니다.

PATIENCE = 7
# PATIENCE: 학습 중에 검증 데이터의 성능이 더 이상 좋아지지 않을 때, 몇 번 더 기다려줄지를 결정하는 값입니다.
# 7로 설정하면 검증 데이터의 성능이 7번 연속으로 좋아지지 않을 때 학습을 조기에 종료합니다.

# np.random.seed(SEED)는 numpy 라이브러리가 사용하는 랜덤(무작위) 숫자 생성의 항상 동일한 결과를 만들기 위한 설정입니다.
# 매개변수:
# - SEED: 랜덤 숫자를 만들 때 사용하는 기준 숫자입니다.
np.random.seed(SEED)

# torch.manual_seed(SEED)는 PyTorch 라이브러리가 사용하는 랜덤 숫자 생성도 항상 동일한 결과가 만들어지도록 설정합니다.
# 매개변수:
# - SEED: 랜덤 숫자 생성의 기준이 되는 숫자입니다.
torch.manual_seed(SEED)

# torch.device("cuda" if torch.cuda.is_available() else "cpu")는 인공지능 모델 학습에 사용할 컴퓨터의 장치를 설정합니다.
# - "cuda" : GPU(그래픽카드)가 있다면 GPU를 사용하도록 자동으로 설정합니다.
# - "cpu"  : GPU가 없으면 CPU를 기본 장치로 설정하여 학습합니다.
# GPU가 있을 경우 훨씬 빠르게 연산이 가능하며, CPU는 비교적 느립니다.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 자바(Java) 프로그램을 사용할 수 있도록 환경 변수를 설정하는 부분입니다.
# 형태소 분석을 컴퓨터가 특정 프로그램을 찾아 실행할 수 있게 경로를 알려주는 작업입니다.

default_jdk = r"C:\Program Files\Java\jdk-21"
# default_jdk는 자바가 설치된 기본 폴더 위치를 지정하는 변수입니다.
# 앞에 r을 붙인 이유는 문자열 그대로 인식하게 하기 위함입니다.

# os.environ은 컴퓨터의 환경 변수를 설정하거나 불러오는 데 사용하는 기능입니다.
# JAVA_HOME 환경변수를 설정합니다.
# 이 변수는 자바 프로그램이 설치된 위치를 나타냅니다.
os.environ['JAVA_HOME'] = default_jdk

# PATH 환경변수를 수정하는 부분입니다.
# 기존 PATH에 자바 실행 파일이 있는 폴더(bin 폴더)를 추가하여
# 컴퓨터가 자바 명령어를 어디서든 실행할 수 있게 해줍니다.
# os.pathsep은 경로들을 구분하기 위한 문자이며 Windows에서는 세미콜론(;)입니다.
# os.path.join(default_jdk, 'bin') : 자바 실행 파일들이 포함된 bin 폴더의 경로를 합친 값입니다.
# os.path.join(default_jdk, 'bin', 'server') : 추가적으로 자바 서버 컴포넌트가 위치한 폴더 경로를 합칩니다.
os.environ['PATH'] += os.pathsep + os.path.join(default_jdk, 'bin') + os.pathsep + os.path.join(default_jdk, 'bin', 'server')

# CSV 파일로부터 데이터를 읽어와서 데이터프레임(표 형태의 데이터)으로 저장하는 과정입니다.
# pd.read_csv() 함수는 CSV파일을 읽어 데이터프레임으로 만들어 줍니다.
# 매개변수:
# - CSV_PATH: 데이터를 읽어올 CSV 파일의 경로(파일 위치와 이름)입니다.
# - encoding: CSV 파일이 저장된 문자형식을 지정합니다. utf-8-sig는 한글 데이터가 포함된 파일을 읽을 때 흔히 사용하는 문자형식입니다.
df = pd.read_csv(CSV_PATH, encoding="utf-8-sig")

df


,Score,Apply,SNS,Addr
0,5,안녕하세요. 데일리 신청해 봅니다. 어렸을 적 부터 장이 좋지 않았는데 고기 인...,twitter,경기도
1,2,오!! 안그래도 장이 안좋아서 아침마다 고생이거든요~~ 먹고 건강해지고 싶네용 ^^,facebook,서울특별시
2,2,요즘 장이 안좋은지 하루종일 더부룩하고 배변후에도 시원하지 않네요.꼭 체험해보고싶습니다.,facebook,서울특별시
3,3,우리 아이가 은가를 동글 동글 염소 똥처럼 눈답니다. 매번 너무 걱정이랍니다. 꼭 ...,facebook,서울특별시
4,5,이것저것 유산균을 먹어봤지만 이거다~ 하는걸 아직 못만났어요. 장이 예민한 우리 작...,kakaostory,경상남도
...,...,...,...,...
2866,3,"하루하루 반복되는 일과, 근무후 회식 및 거래처등과의 저녁약속으로 다음날이면 어김없...",kakaostory,충청북도
2867,2,장이 건강하면 만사가 편하다 해서~,facebook,서울특별시
2868,3,저희부모님이 건강이 좋지않아.. 부모님께한번 드려보고괜찮으면구매해볼까생각중입니다~ ...,facebook,NaN
2869,3,저는 41살의 직장인겸 주부입니다. 15살 중2딸도 함께 체험을 신청합니다. 변비가...,kakaostory,경상북도


In [3]:
# 데이터프레임에서 특정 열에 결측값(NaN, 빈 값)이 있는 행을 삭제하는 과정입니다.
# df.dropna() 함수는 결측값이 있는 데이터를 지워줍니다.
# 매개변수:
# - subset=['Apply']: 'Apply' 열에서만 결측값을 검사하여, 이 열이 비어있으면 해당 행을 삭제합니다.
# - inplace=True: 원본 데이터프레임(df)에 바로 반영(저장)하겠다는 의미입니다.
df.dropna(subset=['Apply'], inplace=True)

df


,Score,Apply,SNS,Addr
0,5,안녕하세요. 데일리 신청해 봅니다. 어렸을 적 부터 장이 좋지 않았는데 고기 인...,twitter,경기도
1,2,오!! 안그래도 장이 안좋아서 아침마다 고생이거든요~~ 먹고 건강해지고 싶네용 ^^,facebook,서울특별시
2,2,요즘 장이 안좋은지 하루종일 더부룩하고 배변후에도 시원하지 않네요.꼭 체험해보고싶습니다.,facebook,서울특별시
3,3,우리 아이가 은가를 동글 동글 염소 똥처럼 눈답니다. 매번 너무 걱정이랍니다. 꼭 ...,facebook,서울특별시
4,5,이것저것 유산균을 먹어봤지만 이거다~ 하는걸 아직 못만났어요. 장이 예민한 우리 작...,kakaostory,경상남도
...,...,...,...,...
2866,3,"하루하루 반복되는 일과, 근무후 회식 및 거래처등과의 저녁약속으로 다음날이면 어김없...",kakaostory,충청북도
2867,2,장이 건강하면 만사가 편하다 해서~,facebook,서울특별시
2868,3,저희부모님이 건강이 좋지않아.. 부모님께한번 드려보고괜찮으면구매해볼까생각중입니다~ ...,facebook,NaN
2869,3,저는 41살의 직장인겸 주부입니다. 15살 중2딸도 함께 체험을 신청합니다. 변비가...,kakaostory,경상북도


In [4]:
# 데이터프레임(df)의 "Apply" 컬럼 값이 같은 중복된 행을 제거합니다.
# drop_duplicates() 함수는 중복된 데이터를 찾아 삭제합니다.
# 매개변수:
# - subset=['Apply']: "Apply"라는 컬럼에서만 중복 여부를 확인합니다.
# - keep='first': 중복된 데이터가 여러 개 있으면 첫 번째 행만 남기고 나머지를 삭제합니다.
# - inplace=True: 이 명령어를 실행한 결과를 바로 원본 데이터프레임(df)에 적용하여 저장합니다.
df.drop_duplicates(subset=['Apply'], keep='first', inplace=True)

# 중복된 데이터가 제대로 제거되었는지 데이터프레임(df)을 출력해서 결과를 확인합니다.
df


,Score,Apply,SNS,Addr
0,5,안녕하세요. 데일리 신청해 봅니다. 어렸을 적 부터 장이 좋지 않았는데 고기 인...,twitter,경기도
1,2,오!! 안그래도 장이 안좋아서 아침마다 고생이거든요~~ 먹고 건강해지고 싶네용 ^^,facebook,서울특별시
2,2,요즘 장이 안좋은지 하루종일 더부룩하고 배변후에도 시원하지 않네요.꼭 체험해보고싶습니다.,facebook,서울특별시
3,3,우리 아이가 은가를 동글 동글 염소 똥처럼 눈답니다. 매번 너무 걱정이랍니다. 꼭 ...,facebook,서울특별시
4,5,이것저것 유산균을 먹어봤지만 이거다~ 하는걸 아직 못만났어요. 장이 예민한 우리 작...,kakaostory,경상남도
...,...,...,...,...
2866,3,"하루하루 반복되는 일과, 근무후 회식 및 거래처등과의 저녁약속으로 다음날이면 어김없...",kakaostory,충청북도
2867,2,장이 건강하면 만사가 편하다 해서~,facebook,서울특별시
2868,3,저희부모님이 건강이 좋지않아.. 부모님께한번 드려보고괜찮으면구매해볼까생각중입니다~ ...,facebook,NaN
2869,3,저는 41살의 직장인겸 주부입니다. 15살 중2딸도 함께 체험을 신청합니다. 변비가...,kakaostory,경상북도


In [5]:
# 삭제 후 데이터프레임의 인덱스(행 번호)를 재설정하는 과정입니다.
# 데이터 행을 삭제한 후에는 인덱스 번호가 중간중간 빠질 수 있으므로 깔끔하게 다시 번호를 매겨줍니다.
# df.reset_index() 함수는 데이터프레임의 행 번호를 다시 0부터 순서대로 설정합니다.
# 매개변수:
# - drop=True: 기존의 인덱스를 새로운 열로 따로 저장하지 않고 완전히 버리겠다는 의미입니다.
# - inplace=True: 원본 데이터프레임(df)에 바로 반영하겠다는 의미입니다.
df.reset_index(drop=True, inplace=True)

df


,Score,Apply,SNS,Addr
0,5,안녕하세요. 데일리 신청해 봅니다. 어렸을 적 부터 장이 좋지 않았는데 고기 인...,twitter,경기도
1,2,오!! 안그래도 장이 안좋아서 아침마다 고생이거든요~~ 먹고 건강해지고 싶네용 ^^,facebook,서울특별시
2,2,요즘 장이 안좋은지 하루종일 더부룩하고 배변후에도 시원하지 않네요.꼭 체험해보고싶습니다.,facebook,서울특별시
3,3,우리 아이가 은가를 동글 동글 염소 똥처럼 눈답니다. 매번 너무 걱정이랍니다. 꼭 ...,facebook,서울특별시
4,5,이것저것 유산균을 먹어봤지만 이거다~ 하는걸 아직 못만났어요. 장이 예민한 우리 작...,kakaostory,경상남도
...,...,...,...,...
2481,3,"하루하루 반복되는 일과, 근무후 회식 및 거래처등과의 저녁약속으로 다음날이면 어김없...",kakaostory,충청북도
2482,2,장이 건강하면 만사가 편하다 해서~,facebook,서울특별시
2483,3,저희부모님이 건강이 좋지않아.. 부모님께한번 드려보고괜찮으면구매해볼까생각중입니다~ ...,facebook,NaN
2484,3,저는 41살의 직장인겸 주부입니다. 15살 중2딸도 함께 체험을 신청합니다. 변비가...,kakaostory,경상북도


In [6]:
# 고객의 평점(Score)을 이용해 열심히 활동(2), 보통(1), 활동 안함(0) 의 세 가지 라벨로 바꿔주는 함수입니다.
# 매개변수:
# - score: 고객 활동의 평점으로, 1점부터 5점까지의 값을 가집니다.
def score_to_label(score):
    # 평점(score)이 3점 미만(1점, 2점)이면 0(활동 안함)을 반환합니다.
    # 평점이 정확히 3점이면 1(보통)을 반환합니다.
    # 평점이 4점 또는 5점이라면 2(열심히 활동)을 반환합니다.
    return 0 if score < 3 else (1 if score == 3 else 2)

# 위에서 만든 score_to_label 함수를 데이터프레임(df)의 'Score' 열에 적용하여 새로운 열(Label)을 만듭니다.
# apply() 함수는 데이터프레임에서 특정 열의 모든 값에 함수를 하나씩 적용하는 기능입니다.
# 예를 들어 데이터프레임(df)의 Score 열 값이 아래와 같다고 가정합니다:
# | Score |
# |-------|
# |   5   |
# |   2   |
# |   3   |
# |   1   |
#
# df['Score'].apply(score_to_label) 실행 결과:
# | Score | Label |
# |-------|-------|
# |   5   |   2   |
# |   2   |   0   |
# |   3   |   1   |
df['label'] = df['Score'].apply(score_to_label)

# 데이터프레임(df)의 내용을 화면에 출력하여 결과를 직접 확인합니다.
df


,Score,Apply,SNS,Addr,label
0,5,안녕하세요. 데일리 신청해 봅니다. 어렸을 적 부터 장이 좋지 않았는데 고기 인...,twitter,경기도,2
1,2,오!! 안그래도 장이 안좋아서 아침마다 고생이거든요~~ 먹고 건강해지고 싶네용 ^^,facebook,서울특별시,0
2,2,요즘 장이 안좋은지 하루종일 더부룩하고 배변후에도 시원하지 않네요.꼭 체험해보고싶습니다.,facebook,서울특별시,0
3,3,우리 아이가 은가를 동글 동글 염소 똥처럼 눈답니다. 매번 너무 걱정이랍니다. 꼭 ...,facebook,서울특별시,1
4,5,이것저것 유산균을 먹어봤지만 이거다~ 하는걸 아직 못만났어요. 장이 예민한 우리 작...,kakaostory,경상남도,2
...,...,...,...,...,...
2481,3,"하루하루 반복되는 일과, 근무후 회식 및 거래처등과의 저녁약속으로 다음날이면 어김없...",kakaostory,충청북도,1
2482,2,장이 건강하면 만사가 편하다 해서~,facebook,서울특별시,0
2483,3,저희부모님이 건강이 좋지않아.. 부모님께한번 드려보고괜찮으면구매해볼까생각중입니다~ ...,facebook,NaN,1
2484,3,저는 41살의 직장인겸 주부입니다. 15살 중2딸도 함께 체험을 신청합니다. 변비가...,kakaostory,경상북도,1


In [7]:
# 데이터프레임(df)의 "label"이라는 열에 있는 값들만 따로 가져와서 y라는 변수에 저장합니다.
# df는 표 형태의 데이터(데이터프레임)으로, 각 열에는 데이터의 종류에 따라 이름이 붙어 있습니다.
# df["label"]는 df라는 데이터프레임의 "label"이라는 이름을 가진 열에 있는 모든 데이터를 선택합니다.
# .values는 선택된 열(Label)의 데이터를 순서대로 뽑아서 배열 형태로 만들어줍니다.
# 예를 들어, df["label"]의 데이터가 아래와 같다고 가정합니다:
#
# | label |
# |-------|
# |   2   |
# |   0   |
# |   1   |
# |   2   |
# |   0   |
#
# 위 표에서 .values를 실행하면 [2, 0, 1, 2, 0] 같은 형태의 배열이 만들어집니다.
# 이렇게 만든 배열을 변수 y에 저장합니다.
y = df["label"].values

# 변수 y를 화면에 출력하여 어떤 값이 저장되어 있는지 확인합니다.
# 위 예시에서는 [2, 0, 1, 2, 0] 과 같은 배열 형태로 보여줍니다.
y


array([2, 0, 0, ..., 1, 1, 1])

In [8]:
# okt는 한글 문장을 형태소(단어의 최소 단위)로 나누는 데 사용하는 도구입니다.
# Okt()는 KoNLPy 라이브러리에서 제공하는 형태소 분석기입니다.
# 예: "제품이 좋다"라는 문장을 넣으면 아래와 같이 형태소와 품사를 분석합니다.
# [('제품', 'Noun'), ('은', 'Josa'), ('좋다', 'Adjective')]
okt = Okt()

# Okt_tokenizer는 형태소를 분석하고,
# 명사(Noun), 형용사(Adjective), 동사(Verb), 부사(Adverb)만 골라주는 함수입니다.
# lambda를 사용하여 간단하게 정의한 형태입니다.
#
# 예를 들어, 입력값: "오늘 제품이 정말 좋다"
# okt.pos() 분석 결과:
# [('오늘', 'Noun'), ('제품', 'Noun'), ('이', 'Josa'), ('정말', 'Noun'),
#  ('좋다', 'Adjective')]
#
# 아래 함수는 이 중 불필요한 조사(Josa)를 제거하고
# Noun, Adjective, Verb, Adverb 만 최종 리스트로 반환합니다.
Okt_tokenizer = lambda text: [w for w, t in okt.pos(str(text), stem=True)
                              if t in ["Noun", "Adjective", "Verb", "Adverb"]]

# 데이터프레임(df)의 'Apply' 열에 있는 모든 문장에 Okt_tokenizer 함수를 적용하여
# 결과를 'tokens'라는 새로운 열로 추가합니다.
#
# 예를 들어 df['Apply']가 아래 형태라면:
# | Apply               |
# |--------------------|
# | "저는 오늘도 잘 구매합니다." |
# | "이 코드는 클린한 이유가 뭐니?"   |
#
# 적용 결과 df['tokens']는 아래와 같습니다:
# | tokens                                    |
# |-------------------------------------------|
# | ['저', '오늘', '구매하다']                 |
# | ['코드', '클린', '이유']                    |
#
df['tokens'] = df['Apply'].apply(Okt_tokenizer)

# 미리 학습된 FastText라는 인공지능 모델을 불러와서 단어를 숫자 벡터로 바꾸는 부분입니다.
# load_facebook_model(FASTTEXT_BIN): 페이스북에서 만든 FastText 모델 파일을 읽어옵니다.
# 매개변수:
# - FASTTEXT_BIN: 사용할 FastText 모델 파일 이름으로 'cc.ko.300.bin'이 들어갑니다.
kv = load_facebook_model(FASTTEXT_BIN).wv

# kv.vector_size: 한 단어가 몇 개의 숫자로 이루어져 있는지를 나타냅니다.
# 예를 들어, '시간'이라는 단어는 [0.12, -0.34, 0.56, ...] 처럼 300개의 숫자로 표현될 수 있습니다.
embedding_dim = kv.vector_size

embedding_dim


300

In [9]:
# vectorize는 문장을 숫자 벡터로 바꾸어주는 함수입니다.
# 각 문장은 여러 단어로 이루어져 있으며, 단어 하나하나를 FastText 모델을 통해 숫자 벡터(300차원)로 변환하여
# 문장을 숫자 배열로 만듭니다.
# 예: ["있었어요", "시간"] 같은 문장은 벡터로 변환하면 [[0.12, -0.34, 0.56, ...], [0.67, 0.89, -0.12, ...]]와 같은 형태로 바뀝니다.
#
# 매개변수 설명:
# - tokens: 토큰화된 문장 리스트입니다. 각 문장은 단어들의 리스트로 구성됩니다. 예: [['있었어요', '시간'], ['가다', '물']]
# - maxlen: 문장을 고정된 길이로 맞춰 줄 때 사용되는 최대 길이입니다. 짧으면 0으로 채우고 길면 자릅니다.
# - kv: FastText 모델의 word vector 객체입니다.
def vectorize(tokens, maxlen, kv):
    # X는 (문장 수, 고정 문장 길이, 단어 벡터 크기)를 갖는 초기 배열을 만듭니다.
    X = np.zeros((len(tokens), maxlen, kv.vector_size))

    # 각 문장의 실제 단어 개수를 저장할 lengths 배열을 만듭니다.
    lengths = np.zeros(len(tokens), int)

    # 모든 문장을 하나씩 살펴봅니다.
    for i, toks in enumerate(tokens):
        # 문장의 길이가 maxlen보다 길면, 뒤쪽 단어는 제거하고 마지막 부분만 남깁니다.
        toks = toks[:maxlen]

        # 현재 문장에서 사용할 벡터 수(단어 개수)를 저장합니다.
        cnt = 0

        # 각 단어를 FastText 벡터로 변환합니다.
        for j, w in enumerate(toks):
            try:
                # kv.get_vector(w): FastText 모델에서 해당 단어(w)의 300차원 벡터를 가져옵니다.
                X[i, j] = kv.get_vector(w)
                cnt += 1
            except KeyError:
                # FastText 모델에 없는 단어면 그냥 넘어갑니다.
                continue

        # 문장의 실제 단어 개수 저장
        lengths[i] = max(1, cnt)

    # 최종적으로 변환된 숫자 벡터 배열(X)과 각 문장의 실제 길이(lengths)를 반환합니다.
    return X, lengths

# 데이터프레임(df)에 저장된 모든 문장(tokens)을 숫자 벡터로 변환합니다.
# 변환된 숫자 벡터는 X_vec이라는 변수에 저장됩니다.
# 각 문장에서 실제로 변환에 성공한 단어의 개수는 Lengths_all 변수에 저장됩니다.

# vectorize() 함수는 문장 속 각 단어를 FastText라는 미리 학습된 모델(kv)을 사용하여 숫자 벡터로 바꿉니다.
# 매개변수 설명:
# - tokens: 토큰화된 각 문장 리스트입니다.
#   예: [["있었어요", "시간"], ["도구", "가다"], ["신선하다", "최고"], ["즐겁다"]]
# - maxlen: 각 문장을 동일한 길이로 맞추기 위한 최대 단어 수입니다.
#   예: maxlen이 3이면 문장이 5단어이면 앞 2개를 잘라내고, 단어가 1개면 2개를 0으로 채워 맞춥니다.
# - kv: FastText 모델로, 각 단어를 300차원 벡터로 변환합니다.
#
# 예를 들어 FastText 모델은 이런 벡터를 반환합니다:
# "있었어요" → [0.12, -0.34, 0.56, ...]  (총 300개 숫자로 표현됨)

# 실험용 간단한 tokens_list가 아래 있다고 가정합니다.
tokens_list = [
    ["있었어요", "시간"],
    ["도구", "가다"],
    ["신선하다", "최고"],
    ["즐겁다"]
]

# 이제 vectorize를 실행해보는 과정을 다음과 같이 굴립니다.
# maxlen은 3이기 때문에 결과는 아래와 같습니다.

# 1번째 문장: ["있었어요", "시간"]
# → 단어 수: 2개
# → 길이가 maxlen보다 짧아서 그대로 사용됩니다.
# → 실제 사용된 단어 개수: 2

# 2번째 문장: ["도구", "가다", "신선하다", "최고"]
# → 단어 수: 4개
# → maxlen(3)를 초과하므로 위에서 3개의 단어만 사용됩니다.
# → 실제 사용된 단어 개수: 3

# 3번째 문장: ["즐겁다"]
# → 단어 수: 1개 (maxlen보다 짧아서 그대로 사용됩니다.)
# → 실제 사용된 단어 개수: 1

# 이렇게 변환된 단어들은 각각 아래와 같이 숫자 벡터로 변환됩니다.
# 예를 들어,
# "있었어요" → [0.31, 0.02, -0.14, ...]   (300개 숫자)
# "시간"     → [-0.22, 0.07, 0.48, ...]   (300개 숫자)
# "도구"     → [0.11, 0.23, -0.34, ...]   (300개 숫자)
# "가다"     → [-0.12, 0.45, 0.01, ...]   (300개 숫자)
# "신선하다" → [-0.15, 0.23, 0.11, ...]  (300개 숫자)
# "최고"     → [0.18, 0.09, 0.45, ...]   (300개 숫자)

# 위 단어 벡터들을 모두 모아 문장 하나당 최대 3개의 단어로 구성된 숫자 벡터가 됩니다.
# 만약 단어가 3개보다 적으면 빈 공간은 모두 0으로 채워집니다.

# 따라서 실제 저장되는 X_vec 같은 아래처럼 됩니다:
# X_vec = [
#   [[0.31, 0.02, -0.14, ...], [-0.22, 0.07, 0.48, ...], [0.0, 0.0, 0.0, ...]],   # 문장 1
#   [[0.11, 0.23, -0.34, ...], [0.18, 0.09, 0.45, ...], [-0.15, 0.27, 0.33, ...]], # 문장 2
#   [[-0.05, -0.22, 0.13, ...], [0.0, 0.0, 0.0, ...], [0.0, 0.0, 0.0, ...]]         # 문장 3
# ]

# 실제 저장되는 lengths_all 값은 다음과 같습니다:
# lengths_all = [2, 3, 1]

# 이제 코드에서 실제 데이터프레임(df)의 tokens을 사용하여 위와 같은 변환을 수행하고 저장합니다.
X_vec, lengths_all = vectorize(df['tokens'], MAX_SEQ_LEN, kv)

X_vec

array([[[-1.91758238e-02, -4.15817089e-03,  1.75054688e-02, ...,
         -2.97760703e-02, -5.38390037e-03, -1.28347864e-02],
        [ 5.89476153e-02,  5.20394333e-02, -1.01685457e-01, ...,
         -9.34071839e-02, -3.23469820e-03,  9.63032022e-02],
        [-1.83001503e-01,  6.12084344e-02,  3.21909525e-02, ...,
         -9.98315066e-02, -2.60102451e-01, -1.13072015e-01],
        ...,
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00]],

       [[-2.22487807e-01, -1.23427180e-03, -1.25260660e-02, ...,
         -3.63667379e-03, -1.87566020e-02,  1.05726667e-01],
        [-1.60257190e-01, -3.05334888e-02,  1.52856022e-01, ...,
         -4.85815331e-02,  1.05347726e

In [10]:
lengths_all

array([40,  9, 12, ..., 17, 36, 33])

In [11]:
# 데이터프레임(df)에서 'label' 열의 값을 따로 떼어 배열(y)에 저장합니다.
# 'label' 값은 각 문장의 활동 점수(활동 안함(0), 보통(1), 열심히 활동함(2))을 나타내는 숫자입니다.
y = df['label'].values

y


array([2, 0, 0, ..., 1, 1, 1])

In [12]:
# 데이터를 훈련(학습), 검증(Validation), 테스트(최종 평가) 용으로 나누는 부분입니다.
# 인공지능이 데이터를 얼마나 잘 학습했고 앞으로 얼마나 일반화할 능력을 갖는지 나누어 평가해야 합니다.
# 이 코드는 데이터를 훈련(Train), 검증(Validation), 테스트(Test) 3개 부분으로 나누게 됩니다.

# 첫 번째로 전체 데이터를 훈련용 데이터와 최종 평가를 위한 테스트 데이터로 나눕니다.
# sklearn의 train_test_split()는 데이터(입력값)를 자동으로 나누어주는 함수입니다.

# - X_vec: 입력 문장들 변환된 문장 데이터입니다. 예: ['제품 좋다', '가격 괜찮네?'] 숫자로 변환된 문장 목록
# - y: 각 문장의 레이블 값입니다. (0, 1, 2) → 활동 안함, 보통, 열심히 활동
# - lengths_all: 각 문장을 구성하는 실제 단어 개수입니다. (RNN 등에서 sequence length로 사용)
# - test_size=0.2: 데이터의 20%를 테스트 데이터로 사용합니다.
# - random_state=SEED: 랜덤으로 나눌 때 결과가 항상 같도록 설정하는 값입니다.
# - stratify=y: 각 레이블의 비율을 유지하면서 균등하게 나눕니다.

X_train, X_test, y_train, y_test, len_train, len_test = train_test_split(
    X_vec, y, lengths_all, test_size=0.2, random_state=SEED, stratify=y
)

# 두 번째로 훈련 데이터를 다시 훈련과 검증 데이터로 나눕니다.
# 검증 데이터는 학습 도중 모델의 성능을 중간 점검하는 데 사용됩니다.

# - X_train: 위에서 나누고 남은 훈련용 데이터입니다.
# - y_train: 그 훈련 데이터의 정답(레이블)입니다.
# - len_train: 각 문장의 단어 개수입니다.
# - test_size=VAL_SIZE: 훈련 데이터의 10%만 검증 데이터로 사용하는 설정입니다.
# - random_state=SEED: 실험 결과를 고정하기 위함입니다.
# - stratify=y_train: 레이블 비율을 유지한 상태로 균등하게 나눕니다.

X_tr, X_val, y_tr, y_val, len_tr, len_val = train_test_split(
    X_train, y_train, len_train, test_size=VAL_SIZE, random_state=SEED, stratify=y_train
)


In [13]:
# 데이터를 로더(Dataloader)로 만드는 함수(get_loader)를 정의합니다.
# 데이터 로더는 데이터를 미니배치(mini-batch)라는 작은 묶음으로 만들어
# 인공지능 모델이 쉽게 학습하도록 도와주는 도구입니다.
#
# 미니배치는 한 번에 정해진 크기(BATCH_SIZE)의 데이터를 모아서 넣는 방식입니다.
#
# 매개변수:
# - X: 숫자 데이터(문장 벡터 형태의 문장들)
# - lens: 각 문장의 길이 정보(단어 개수)
# - y: 각 문장의 레이블(정답)
# - shuffle: 데이터를 섞을지 여부를 정합니다. 훈련 시에는 True, 검증과 테스트 시에는 False를 씁니다.
get_loader = lambda X, lens, y, shuffle: data.DataLoader(
    data.TensorDataset(
        torch.tensor(X).float(), 
        torch.tensor(lens),      
        torch.tensor(y)
    ),
    BATCH_SIZE,
    shuffle
)

# 위에서 정의한 get_loader 함수를 사용하여 실제 데이터 로더를 만듭니다.
# 훈련, 검증, 테스트 각각의 데이터를 로더로 만들어 모델에 넘겨줍니다.

train_loader = get_loader(X_tr, len_tr, y_tr, True)     # 훈련 데이터 → 섞음
val_loader   = get_loader(X_val, len_val, y_val, False) # 검증 데이터 → 섞지 않음
test_loader  = get_loader(X_test, len_test, y_test, False) # 테스트 데이터 → 섞지 않음


In [14]:
# SimpleRNNClassifier는 가장 기본적인 형태의 RNN(순환 신경망)을 사용하여
# 문장(문장이 어떤 활동 정도인지 예측), 활동 분류를 위한 모델입니다.
# 입력으로 단어 벡터(embedding) 순차열을 입력받아 RNN을 통과해 마지막 hidden 상태(h)를 얻고,
# 최종 선형 레이어(fc)를 지나 활동 분류(0,1,2)를 수행할 수 있습니다.

class SimpleRNNClassifier(nn.Module):
    """
    RNN 기반 간단한 분류 모델을 정의한 클래스입니다.
    """
    def __init__(self, emb_dim, hid_dim, num_classes):
        super().__init__()
        # emb_dim: 단어 하나를 표현할 때 사용하는 숫자의 갯수입니다. 예를 들어 300이면 단어 하나가 300 개의 숫자로 표현됩니다.
        # hid_dim: RNN 내부에서 사용하는 숨겨진 차원의 크기입니다. 예를 들어 64 라면 64 차원의 숫자로 기억합니다.
        # num_classes: 분류하고 싶은 클래스의 수입니다. 여기서는 3가지(0,1,2).
        self.rnn = nn.RNN(emb_dim, hid_dim, batch_first=True)
        # nn.RNN: 가장 기본적인 RNN 모델을 만들어줍니다.
        # emb_dim: 입력 단어의 크기입니다.
        # hid_dim: RNN이 내부에서 계산하는 숨겨진 차원의 길이입니다.
        # batch_first=True: 미니배치 (문장 묶음)가 첫 번째 축이라는 의미입니다.

        self.fc = nn.Sequential(
            nn.Dropout(0.5), # nn.Dropout은 모델이 특정 특성에 과적합되는 것을 막기 위해 일부 뉴런을 무작위로 제거합니다. 0.5는 50%를 의미합니다.
            nn.Linear(hid_dim, num_classes) # nn.Linear는 출력층이며, 입력은 숨겨진 차원, 출력은 3개 활동 클래스를 예측합니다.
        )

    def forward(self, x, lens):
        """
        forward는 실제로 데이터를 넣어서 예측을 진행하는 함수입니다.
        x: 입력 문장 데이터들. (문장 개수, 문장 길이, 단어 벡터 크기)
        lens: 각 문장의 실제 길이입니다. ["좋아요", "가성비 최고!"] 같은 경우 단어 수에 따라 길이가 달라집니다.
        """
        h, _ = self.rnn(x)  # h는 모든 단어의 숨겨진 상태들을 담습니다. (문장 길이마다 하나씩.)
        # 마지막 단어의 hidden state(h)를 선택합니다. h[배치_인덱스, 마지막_위치]
        # lens는 각각의 문장 길이를 담고 있어, 마지막 단어의 위치를 선택할 수 있게 합니다.
        last_h = h[torch.arange(len(lens)), lens - 1, :]  # 각 문장마다 마지막 단어 위치의 h(state)를 모아줍니다.
        return self.fc(last_h) # 마지막 단어의 hidden state를 fully-connected에 넣어 예측 값을 반환합니다.

# simple_rnn_model은 인공지능 모델 중에서 가장 기본적인 형태인 RNN(순환 신경망) 모델입니다.
# 이 모델은 여러 문장을 읽고, 각 문장이 총 어떤 활동 상태에 속하는지 판단합니다:
# 0: 활동 불만 요소, 1: 활동 보통 상태, 2: 활동 매우 열심히 함
# (예를 들어 "이 제품 정말 좋아요!"라는 리뷰를 입력받으면 모델이 '활동 매우 열심히 함(2)'으로 예측할 수 있습니다.

# SimpleRNNClassifier 모델을 생성합니다.
# 이 클래스는 리뷰 문장을 입력받으면 활동 상태를 판단하는 역할을 합니다.
# 매개변수 설명:
# - embedding_dim: 각 단어를 표현할 때 사용하는 숫자 벡터의 길이입니다.
#   예를 들어 300이면, '좋다'라는 단어는 [0.1, 0.2, ..., -0.3]처럼 길이 300개의 숫자로 표현됩니다.
# - hid_dim: RNN 내부 모델 내부에서 계산할 때 사용하는 벡터의 크기입니다.
#   이 값을 크게 할수록 모델이 더 복잡한 패턴을 기억합니다. 예시로는 64와 같은 숫자를 주로 사용합니다.
# - num_classes: 예측할 활동의 개수로, 여기서는 3개입니다(불만(0), 보통(1), 열심히 함(2)).
simple_rnn_model = SimpleRNNClassifier(embedding_dim, 64, 3)

# 모델의 구조를 확인하기 위해 화면에 출력합니다.
# 실행하면 아래와 같은 구조가 나옵니다:
# 예:
# SimpleRNNClassifier(
#   (rnn): RNN(300, 64, batch_first=True)
#   (fc): Sequential(
#     (0): Dropout(p=0.5, inplace=False)
#     (1): Linear(in_features=64, out_features=3, bias=True)
#   )
# )
# 여기서 RNN(300, 64)는 단어를 표현하는 숫자가 300개이고, 문장을 기억할 때 64개의 숫자를 사용한다는 의미입니다.
simple_rnn_model


SimpleRNNClassifier(
  (rnn): RNN(300, 64, batch_first=True)
  (fc): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=64, out_features=3, bias=True)
  )
)

In [15]:
# 앞에서 만든 simple_rnn_model 이는 인공지능 모델을 GPU 또는 CPU에서 사용 가능하게 설정합니다.
# 모델이 GPU를 사용할 수 있으면 빠르게 처리할 수 있고, GPU가 없으면 CPU를 사용합니다.
# 매개변수:
# - device: 모델을 실행할 장치로 GPU(cuda) 또는 CPU(cpu)입니다.
simple_rnn_model = simple_rnn_model.to(device)

# 손실함수(loss function)를 설정합니다.
# 손실함수는 인공지능이 예측한 결과와 실제 정답의 차이를 계산해서, 얼마나 잘못 예측했는지 숫자로 알려주는 함수입니다.
# 여기서 사용하는 손실함수는 CrossEntropyLoss로, 주로 분류 문제(여러가지 클래스 중 하나를 맞추는 문제)에 사용됩니다.
# 활동 활동 [고양이, 강아지, 보통] 도 같이 시간에 있는 동물로 맞추는 문제처럼 각 문서 예측을 할 때 얼마나 틀렷는지 계산해줍니다.
criterion = nn.CrossEntropyLoss()

# 옵티마이저(optimizer)는 인공지능 모델이 학습할 때 모델 내부의 값들(가중치)을 조금씩 조정해서 더 좋은 결과를 얻도록 도와주는 역할을 합니다.
# 여기서 사용하는 옵티마이저는 Adam이라는 방법입니다. Adam은 일반적으로 좋은 결과를 얻도록 도와주는 인기있는 학습 방법입니다.
#
# 매개변수:
# - simple_rnn_model.parameters(): 인공지능 모델의 학습될 때 변경되는 가중치(내부 숫자들)입니다.
# - lr=LR: Learning Rate(학습률): 학습할 때 숫자를 얼마나 크게 바꿀지 정하는 숫자입니다. 너무 크면 학습이 잘 안되고, 너무 작으면 학습이 느려집니다.
# - weight_decay=WEIGHT_DECAY: 가중치가 너무 커지거나 너무 작지 않게 모델이 흔들리지 않도록 데이터에서도 잘 작동하도록 도와주는 숫자입니다.
optimizer = optim.Adam(simple_rnn_model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)

# 학습 스케줄러(scheduler)는 학습을 진행하면서 학습률(Lr)을 자동으로 줄여주는 역할을 합니다.
# 이를 사용하는 이유는 학습을 하다보면 어떤 지점 이후에는 성능이 좋아지지 않을 때, 학습률을 줄여서 조금 더 섬세하게 학습하도록 돕기 위함입니다.
# 여기서는 ReduceLROnPlateau라는 함수를 사용합니다.
# 매개변수:
# - optimizer: 위에서 만든 옵티마이저입니다.
# - patience=5: 성능이 개선되지 않고 연속 5번 동안 개선되지 않으면 학습률을 줄입니다.
# - factor=0.9: 학습률을 줄일 때 현재 값의 90%로 줄입니다. 즉, 조금 줄이는 방식입니다.
# - min_lr=1e-6: 학습률이 아무리 줄어들어도 0에 가까운 값이 되지는 않도록 바닥값을 설정합니다. (1e-6은 0.000001 입니다.)
scheduler = ReduceLROnPlateau(optimizer, patience=5, factor=0.9, min_lr=1e-6)

# 가장 좋은 손실값(best_loss)을 저장하는 변수로, 처음에는 무한대로 설정합니다.
# 학습하면서 더 작은 손실값이 나오면 여기에 저장합니다.
best_loss = float('inf')

# patience_cnt는 성능이 연속적으로 좋아지지 않은 횟수를 세는 변수입니다.
# 이 횟수가 일정 이상이면 학습을 더 이상 하지 않고 중단합니다.
patience_cnt = 0

# 가장 성능이 좋았던 인공지능 모델의 상태(가중치 값들)를 저장하는 변수입니다.
# 처음에는 아무것도 저장되어 있지 않습니다.
best_state = None


In [16]:
# 모델을 학습시키고 검증하는 전체 과정입니다.
# 학습은 EPOCHS번 반복하며, 한 번의 epoch는 전체 데이터를 한 번씩 보는 것입니다.
# 매 epoch마다 훈련 데이터를 사용해 모델을 학습하고, 검증 데이터를 사용해 모델 성능을 평가합니다.
print(f"\nTraining")

# range(EPOCHS)는 0부터 EPOCHS-1까지 반복입니다. 예: EPOCHS=100이면 0부터 99까지 100회 반복입니다.
for epoch in range(EPOCHS):

    # simple_rnn_model.train()은 모델을 학습 모드로 바꿉니다. 드롭아웃(dropout) 같은 학습 시만 사용하는 기능을 활성화 합니다.
    simple_rnn_model.train()

    # loss_total은 이번 epoch의 전체 훈련 데이터에서 발생한 손실값을 저장하는 변수입니다.
    loss_total = 0

    # train_loader는 앞에서 만든 훈련 데이터 로더입니다.
    # 한 번에 16개씩 데이터를 꺼내 xb(문장 벡터), lb(문장 길이), yb(정답 라벨)에 저장합니다.
    for xb, lb, yb in train_loader:

        # optimizer.zero_grad()는 이전 단계의 기울기(gradient)를 초기화하는 작업입니다.
        # 기울기는 각 가중치가 어떻게 조절되어야 할지 알려주는 값으로, 매번 초기화해야 합니다.
        optimizer.zero_grad()

        # logits는 모델이 데이터를 보고 예측한 결과입니다. (예측값)
        # 매개변수 설명:
        # - xb.to(device): 입력 문장 벡터를 모델이 실행 중인 장치(GPU 또는 CPU)로 이동시킵니다.
        # - lb.to(device): 문장의 길이 데이터도 동일하게 GPU나 CPU로 이동시킵니다.
        logits = simple_rnn_model(xb.to(device), lb.to(device))

        # 손실(loss)을 계산합니다. 손실값은 예측한 값과 실제 정답의 차이 정도를 숫자로 나타낸 것입니다.
        # 매개변수 설명:
        # - logits: 모델의 예측값 입니다.
        # - yb.to(device): 실제 정답 라벨입니다. 역시 GPU 또는 CPU로 이동시킵니다.
        loss = criterion(logits, yb.to(device))

        # loss.backward()는 기울기(gradient)를 계산합니다. 이 과정은 각 가중치가 어떻게 변경되어야 손실이 줄어드는지 알아냅니다.
        loss.backward()
        
        # clip_grad_norm_ 함수는 기울기의 크기를 제한하는 기능입니다.
        # 기울기가 너무 크면 가중치가 너무 많이 변경돼 학습이 불안정해질 수 있으므로 제한해줍니다.
        # 매개변수:
        # - simple_rnn_model.parameters(): 기울기를 제한할 대상 가중치입니다.
        # - CLIP_NORM: 기울기의 최대 크기입니다. 이 값보다 크면 자동으로 줄여줍니다.
        nn.utils.clip_grad_norm_(simple_rnn_model.parameters(), CLIP_NORM)
        
        # optimizer.step()은 계산된 기울기를 바탕으로 실제로 가중치를 조금씩 조정하여 손실을 줄여나갑니다.
        optimizer.step()
        
        # 이번 미니 배치의 손실값을 전체 손실값에 더합니다.
        loss_total += loss.item()

    # 이번 epoch의 검증 데이터를 이용해 모델 성능을 측정합니다.
    # val_loss는 검증 데이터에서 발생한 손실값을 저장하는 변수입니다.
    val_loss = 0
    
    # simple_rnn_model.eval()은 모델을 평가 모드로 바꿉니다. 평가 시에는 드롭아웃 등의 기능을 끕니다.
    simple_rnn_model.eval()
    
    # torch.no_grad()는 평가 시 기울기를 계산하지 않고 메모리를 절약하도록 합니다.
    with torch.no_grad():
    
        # 검증 데이터를 한 번에 16개씩 불러와서 사용합니다.
        for xb, lb, yb in val_loader:
    
            # 검증 데이터의 예측값을 얻습니다.
            logits = simple_rnn_model(xb.to(device), lb.to(device))
    
            # 검증 데이터의 손실을 계산하고 더합니다.
            val_loss += criterion(logits, yb.to(device)).item()
    
        # 전체 검증 데이터의 평균 손실값을 계산합니다.
        val_loss /= len(val_loader)
    
    # scheduler.step()은 검증 손실이 더 이상 개선되지 않을 때 학습률(Lr)을 자동으로 낮춥니다.
    scheduler.step(val_loss)
    
    # epoch 번호와 검증 손실을 화면에 출력합니다.
    print(f"Epoch{epoch+1} Train:{loss_total/len(train_loader):.4f} Val:{val_loss:.4f}")
    
    # 만일 검증 손실이 지금까지의 최고(best_loss)보다 낮으면, 즉 성능이 개선되었으면,
    if val_loss < best_loss:
        best_loss = val_loss           # 최고 손실을 업데이트합니다.
        best_state = simple_rnn_model.state_dict()   # 현재 모델의 가중치를 저장합니다.
        patience_cnt = 0               # 성능 개선을 기다리는 횟수를 초기화합니다.
    else:
        patience_cnt += 1              # 성능이 이번 epoch에서 개선되지 않았으면 기다리는 횟수를 1 증가시킵니다.
    
    # patience_cnt가 설정한 PATIENCE(예:7)을 넘으면, 더 이상 성능 개선이 없다고 판단하여 학습을 중단합니다.
    if patience_cnt >= PATIENCE:
        print(f"Early stopping!")
        break  # 반복(for)을 종료하고 학습을 종료합니다.



Training
Epoch1 Train:0.8944 Val:0.7893
Epoch2 Train:0.7296 Val:0.7179
Epoch3 Train:0.6888 Val:0.6434
Epoch4 Train:0.6562 Val:0.6824
Epoch5 Train:0.5833 Val:0.7374
Epoch6 Train:0.5708 Val:0.7230
Epoch7 Train:0.5686 Val:0.7351
Epoch8 Train:0.5438 Val:0.6346
Epoch9 Train:0.5272 Val:0.6466
Epoch10 Train:0.5047 Val:0.6337
Epoch11 Train:0.5180 Val:0.6556
Epoch12 Train:0.5190 Val:0.6217
Epoch13 Train:0.4771 Val:0.7828
Epoch14 Train:0.4859 Val:0.7225
Epoch15 Train:0.4485 Val:0.7056
Epoch16 Train:0.4166 Val:0.7366
Epoch17 Train:0.4316 Val:0.7446
Epoch18 Train:0.4001 Val:0.8278
Epoch19 Train:0.3625 Val:0.8317
Early stopping!


In [17]:
# 학습 종료 후, 가장 성능이 좋았던 시점(best_state)의 모델 상태를 불러옵니다.
# 즉, 최고 성능을 낸 상태의 모델로 되돌립니다.
simple_rnn_model.load_state_dict(best_state)

# 훈련이 끝난 모델을 사용하여 테스트 데이터에 대한 성능을 측정합니다.
# 예를 들어, 테스트 문장이 아래와 같은 리스트로 있을 때,
# - ["이 제품 좋아요"], ["가성비 좋음"], ["배송이 빨랐음"] 등등처럼
# 각 문장이 어떤 활동 상태인지(활동안함(0), 보통 활동(1), 활발 활동(2) 중 어떤 것인지 예측하여 정확도를 평가합니다.
simple_rnn_model.eval()   # 모델을 평가 모드로 전환합니다. 이런 평가 중에는 쓰는 일부 기능(드롭아웃 등)이 비활성화됩니다.
y_pred = []                # 예측된 클래스(레이블)를 담기 위한 리스트입니다.

# torch.no_grad()는 평가 시에는 그라디언트(기울기)를 계산하지 않아 속도와 메모리를 절약하는 기능입니다.
with torch.no_grad():
    # 테스트 데이터를 한 번에 미니배치(16개씩)로 나누어서 사용합니다.
    # xb는 문장의 벡터화된 데이터가 담겨 있고,
    # lb는 문장의 길이 텐서가 담겨 있고,
    # yb는 실제 정답(레이블)이 담겨 있습니다. 여기서는 사용하지 않고 정확도 계산만 합니다.
    for xb, lb, _ in test_loader:
        # simple_rnn_model(xb.to(device), lb.to(device)): 모델에 데이터를 넣고 예측합니다.
        # xb = 입력 데이터로, 예를 들어 ["가성비 좋다"], ["제품 또봄"] 같은 문장을 벡터로 변환한 데이터입니다.
        logits = simple_rnn_model(xb.to(device), lb.to(device))

        # logits.argmax(1): 모델의 예측 중에서 가장 가능성이 높은 클래스를 선택합니다.
        # 예를 들어 [0.1, 0.2, 0.7]이면 가장 높은 값(0.7)을 가진 2번 클래스를 선택합니다.
        y_pred.extend(logits.argmax(1).cpu().numpy())

# 평가 결과를 화면에 출력하는 부분입니다.
print(f"\n📌 모델 평가 결과")

# accuracy_score(y_test, y_pred)는 실제 정답(y_test)과 모델이 예측한 값(y_pred)을 비교하여 정확도를 계산합니다.
# 예시: 실제 정답이 [0, 1, 2], 모델 예측값이 [0, 2, 2]라면 2/3으로 약 66.7%의 정확도가 나옵니다.
print("정확도 (Accuracy):", accuracy_score(y_test, y_pred))



📌 모델 평가 결과
정확도 (Accuracy): 0.6827309236947792


In [18]:
# confusion_matrix(y_test, y_pred)는 실제 정답과 예측값의 각 조합이 얼마나 자주 나타나는지 보여줍니다.
# 예시: [[5,1,0],[2,4,1],[0,1,6]]처럼 각 클래스 간 혼동된 횟수를 알려줍니다.
print("혼동 행렬 (Confusion Matrix):\n", confusion_matrix(y_test, y_pred))


혼동 행렬 (Confusion Matrix):
 [[ 53  26   1]
 [ 28 229  43]
 [  2  58  58]]


In [19]:
# classification_report(y_test, y_pred, target_names=['활동 안함(0)', '활동 보통(1)', '열심히 활동함(2)'])는
# 각 클래스별로 정밀도(precision), 재현율(recall), F1 점수를 보여줍니다.
# 예를 들어, 활동 안함 클래스의 모델은 활동 안함으로 예측한 것 중 실제 활동 안함 비율을 나타냅니다.
print(classification_report(y_test, y_pred, target_names=['홍보 활동 안함 (0)', '홍보 활동 보통 (1)', '홍보 활동 열심히 함 (2)']))


                 precision    recall  f1-score   support

   홍보 활동 안함 (0)       0.64      0.66      0.65        80
   홍보 활동 보통 (1)       0.73      0.76      0.75       300
홍보 활동 열심히 함 (2)       0.57      0.49      0.53       118

       accuracy                           0.68       498
      macro avg       0.65      0.64      0.64       498
   weighted avg       0.68      0.68      0.68       498



In [20]:
def predict_sentence(model, sentence, tokenizer, kv, maxlen, device):
    # 사용자 입력 문장을 토큰화
    tokens = tokenizer(sentence)

    # 숫자 벡터로 변환
    X, lengths = vectorize([tokens], maxlen, kv)

    # tensor로 변환
    X_tensor = torch.tensor(X).float().to(device)
    lengths_tensor = torch.tensor(lengths).to(device)

    # 모델 평가 모드
    model.eval()
    with torch.no_grad():
        logits = model(X_tensor, lengths_tensor)
        predicted_label = logits.argmax(1).item()

    # 레이블 숫자를 의미 있는 문구로 변환
    labels_meaning = {
        0: "홍보 활동 안함 (0)",
        1: "홍보 활동 보통 (1)",
        2: "홍보 활동 열심히 함 (2)"
    }

    return labels_meaning[predicted_label]



# 사용자가 실제로 문장을 입력하면, 그 입력 내용을 받아서 모델이 예측한 결과를 화면에 보여주는 코드입니다.
# 예시로 입력할 문장: "이 제품이 정말 좋아요!", "별로 효과가 없네요"

while True:
    # 사용자가 분석할 문장을 입력합니다.
    user_input = input("\n분석할 문장을 입력하세요 (종료하려면 'exit' 입력): ")
    
    # 만약 사용자가 "exit"를 입력하면 반복을 중단하고 종료합니다.
    if user_input.lower() == 'exit':
        break
    
    # 사용자가 입력한 문장(user_input)을 실제로 분석하여 예측 결과를 얻습니다.
    prediction = predict_sentence(simple_rnn_model, user_input, Okt_tokenizer, kv, MAX_SEQ_LEN, device)
    
    # 분석 결과를 출력합니다.
    # 예시: 입력 문장: "이 제품이 정말 좋아요!" → 출력 결과: "홍보 활동 열심히 함 (2)"
    print(f"입력 문장 예측 결과: {prediction}")



분석할 문장을 입력하세요 (종료하려면 'exit' 입력):  체험해보고 효과를 느끼면 블로그·SNS에 후기 남기고, 가족에게도 추천하고 싶다


입력 문장 예측 결과: 홍보 활동 보통 (1)



분석할 문장을 입력하세요 (종료하려면 'exit' 입력):  여러 가지 유산균/프로바이오틱스를 꾸준히 먹어봤지만 가격·효과 면에서 아쉬웠다


입력 문장 예측 결과: 홍보 활동 보통 (1)



분석할 문장을 입력하세요 (종료하려면 'exit' 입력):  이번 제품은 장까지 살아서 간다는 코팅 기술과 믿을 수 있는 성분 구성이 눈에 띄어 꼭 체험해보고 싶습니다.  지금까지 사용했던 제품들과 어떤 차이가 있을지 직접 비교해보고 싶고, 효과가 있다면 꾸준히 복용하면서 가족들에게도 추천하고 싶습니다.  체험하게 된다면 솔직하고 자세한 사용 후기를 블로그와 SNS에 정성스럽게 남기겠습니다.  일상 변화와 장 건강에 도움이 되는지 기록하면서 많은 분들에게도 도움이 되도록 후기 공유하겠습니다.


...


분석할 문장을 입력하세요 (종료하려면 'exit' 입력):  ㄱㄱㄱㄱㄱㄱㄱㄱㄱ


입력 문장 예측 결과: 홍보 활동 안함 (0)



분석할 문장을 입력하세요 (종료하려면 'exit' 입력):  좋아요


입력 문장 예측 결과: 홍보 활동 안함 (0)


KeyboardInterrupt: Interrupted by user

In [ ]:
Kernel → Restart Kernel